In [1]:
import pandas as pd
import numpy as np
import datetime
import gc


In [2]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

In [3]:
def fix_str_float(ds, col):
    ds[col] = ds[col].str.replace(r'[^0-9\.]','')
    ds[col] = np.where(ds[col]=='',np.nan,ds[col])
    ds[col] = ds[col].astype('float32')
    return ds[col].astype('float32')

In [4]:
def remove_unused_cols(X):
    #remove empty columns
    to_drop = []
    for col in X.select_dtypes(include='number').columns:
        if X[col].sum()==0:
            to_drop.append(col)
    if len(to_drop)>0:
        X = X.drop(columns=to_drop)
    return X

In [5]:
def preprocess(self, X , y = None ):
    # X = X.copy() # do we need this line
    # boolean transformations
    if 'event_uuid' in X.columns:
        X['event_uuid'] = np.where(X['event_uuid'].isnull(), 0,1)
    if 'click_hash' in X.columns:
        X['click_hash'] = np.where(X['click_hash'].isnull(), 0,1)
    if 'Android' in X.columns:
        X['Android'] = np.where(X['user_agent'].str.contains('Android', regex=False),1,0)
    if 'iOS' in X.columns:
        X['iOS'] = np.where(X['user_agent'].str.contains('Darwin', regex=False) | X['user_agent'].str.contains('iOS', regex=False),1,0)
    if 'trans_id' in X.columns:
        X['trans_id'] = np.where(X['trans_id'].isnull(), 0,1)
    # date transformations
    if 'created' in X.columns:
        X['created_weekday'] = X['created'].dt.weekday
        X['created_hour'] = X['created'].dt.hour
        X['created_minute'] = X['created'].dt.minute
    if 'date' in X.columns:
        X['date_weekday'] = X['date'].dt.weekday
        X['date_hour'] = X['date'].dt.hour
        X['date_minute'] = X['date'].dt.minute
        X['date_second'] = X['date'].dt.second
    #remove unused columns
    to_drop = []
    for col in ['date','created', 'install_diff','device_brand','install_seconds','user_agent','device_id']:
        if col in X.columns:
            to_drop.append(col)
    X = X.drop(columns=to_drop)
    X = pd.get_dummies(X, dummy_na=True, prefix_sep='=')
    #returns numpy array
    return X

    

In [6]:
#read from sample
convertions_ids_sample = pd.read_csv('data/convertions_ids_sample.csv', dtype={'ref_hash':'object'})['ref_hash']

In [7]:
# auctions
#auctions_df = pd.read_csv('data/auctions.csv', low_memory=False, dtype={'country':'category','platform':'category',\
#                                                                        'ref_type_id':'category','source_id':'category','device_id':'object'})
#auctions_df['date'] = pd.to_datetime(auctions_df['date'])
#auctions_sample = auctions_df.loc[auctions_df['device_id'].isin(auctions_ids_sample)].copy()
# installs
installs_df = pd.read_csv('data/installs.csv', low_memory=False, dtype={'ref_type':'category','application_id':'category',\
                                                      'device_brand':'category','ref_hash':'object','wifi':'category'})
installs_df['kind'] = installs_df['kind'].str.lower()
installs_df['kind'] = installs_df['kind'].astype('category')
installs_df.drop(columns=['session_user_agent','ip_address','device_language','device_model'], inplace=True)
installs_df['created'] = pd.to_datetime(installs_df['created'])
installs_df.drop(['device_countrycode'], axis=1, inplace=True)
installs_sample = installs_df.loc[installs_df['ref_hash'].isin(convertions_ids_sample)].copy()
#events
events_df = pd.read_csv('data/events.csv', low_memory=False, dtype={'event_id':'int32','ref_type':'category','application_id':'category',\
                                                                                            'attributed':'bool','device_countrycode':'category','device_city':'category',\
                                                                                            'trans_id':'category','carrier':'category','device_os':'category',\
                                                                                            'connection_type':'category'})
events_df['date'] = pd.to_datetime(events_df['date'])
events_df['wifi'].astype('bool', inplace=True)
events_df.drop(columns=['device_countrycode','session_user_agent','ip_address','device_language'], inplace=True)
events_sample = events_df.loc[events_df['ref_hash'].isin(convertions_ids_sample)].copy()
del events_df
#del auctions_df
del installs_df
gc.collect()
allDone()
print('setup done')

setup done


In [8]:
installs_sample.shape

(4151, 13)

In [9]:
events_sample.shape

(67422, 19)

In [10]:
installs_sample.dtypes

created           datetime64[ns]
application_id          category
ref_type                category
ref_hash                  object
click_hash                object
attributed                  bool
implicit                    bool
device_brand            category
user_agent                object
event_uuid                object
kind                    category
wifi                    category
trans_id                  object
dtype: object

In [11]:
installs_sample.dtypes

created           datetime64[ns]
application_id          category
ref_type                category
ref_hash                  object
click_hash                object
attributed                  bool
implicit                    bool
device_brand            category
user_agent                object
event_uuid                object
kind                    category
wifi                    category
trans_id                  object
dtype: object

In [12]:
# calculate time in seconds
installs_sample.drop_duplicates(inplace=True)
installs_sample = installs_sample.sort_values(by=['ref_hash','created'])
installs_sample['date_dif'] = installs_sample['created'].shift(periods=-1) - installs_sample['created']
installs_sample['device_id_next'] = installs_sample['ref_hash'].astype('object').shift(periods=-1)
installs_sample['date_dif'] = pd.to_timedelta(np.where(installs_sample['device_id_next']==installs_sample['ref_hash'], installs_sample['date_dif'], np.where(installs_sample['created']+pd.DateOffset(3)>'2019-04-27 00:00:00', datetime.datetime(2019,4,27)-installs_sample['created'], pd.to_timedelta(3, unit='d'))))
installs_sample['in_seconds'] = installs_sample['date_dif'].dt.total_seconds()
installs_sample['status_censored'] = ((installs_sample['device_id_next']==installs_sample['ref_hash']) & (installs_sample['in_seconds']<259200.0))
installs_sample.drop(['device_id_next','date_dif'], axis='columns', inplace=True)
installs_sample['ref_hash'] = installs_sample['ref_hash'].astype('object')
#calculate previous time in seconds
installs_sample['date_prev'] = installs_sample['created'].shift()
installs_sample['date_dif_prev'] = installs_sample['created']- installs_sample['date_prev']
installs_sample['device_id_prev'] = installs_sample['ref_hash'].astype('object').shift()
installs_sample['date_dif_prev'] = pd.to_timedelta(np.where(installs_sample['device_id_prev']==installs_sample['ref_hash'], installs_sample['date_dif_prev'], np.where(installs_sample['created']-pd.DateOffset(3)<'2019-04-18 00:00:00', installs_sample['created']-datetime.datetime(2019,4,18), pd.to_timedelta(3, unit='d'))))
installs_sample['last_seen'] = installs_sample['date_dif_prev'].dt.total_seconds()
installs_sample.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
installs_sample = installs_sample.sort_values(by=['created'])

#average time
installs_sample['hora'] = installs_sample['created'].dt.hour
installs_sample['dia'] = installs_sample['created'].dt.day
pivot_auctions = installs_sample.groupby(['dia','hora']).size().reset_index()
pivot_auctions.columns = ['dia','hora', 'hour_day_dist']
max_auctions = pivot_auctions['hour_day_dist'].max().max()
pivot_auctions['hour_day_dist'] = pivot_auctions['hour_day_dist']/max_auctions
display(pivot_auctions.head(5))
installs_sample = pd.merge(installs_sample, pivot_auctions, on=['dia','hora'], how='left')
installs_sample.drop(columns=['hora','dia'], inplace=True)




,dia,hora,hour_day_dist
0,18,0,0.479167
1,18,1,0.791667
2,18,2,0.687500
3,18,3,0.416667
4,18,4,0.395833


In [13]:
# calculate time in seconds
events_sample.drop_duplicates(inplace=True)
events_sample = events_sample.rename(columns = {'date':'created'}).sort_values(by=['ref_hash','created'])
events_sample['date_dif'] = events_sample['created'].shift(periods=-1) - events_sample['created']
events_sample['device_id_next'] = events_sample['ref_hash'].astype('object').shift(periods=-1)
events_sample['date_dif'] = pd.to_timedelta(np.where(events_sample['device_id_next']==events_sample['ref_hash'], events_sample['date_dif'], np.where(events_sample['created']+pd.DateOffset(3)>'2019-04-27 00:00:00', datetime.datetime(2019,4,27)-events_sample['created'], pd.to_timedelta(3, unit='d'))))
events_sample['in_seconds'] = events_sample['date_dif'].dt.total_seconds()
events_sample['status_censored'] = ((events_sample['device_id_next']==events_sample['ref_hash']) & (events_sample['in_seconds']<259200.0))
events_sample.drop(['device_id_next','date_dif'], axis='columns', inplace=True)
events_sample['ref_hash'] = events_sample['ref_hash'].astype('object')
#calculate previous time in seconds
events_sample['date_prev'] = events_sample['created'].shift()
events_sample['date_dif_prev'] = events_sample['created']- events_sample['date_prev']
events_sample['device_id_prev'] = events_sample['ref_hash'].astype('object').shift()
events_sample['date_dif_prev'] = pd.to_timedelta(np.where(events_sample['device_id_prev']==events_sample['ref_hash'], events_sample['date_dif_prev'], np.where(events_sample['created']-pd.DateOffset(3)<'2019-04-18 00:00:00', events_sample['created']-datetime.datetime(2019,4,18), pd.to_timedelta(3, unit='d'))))
events_sample['last_seen'] = events_sample['date_dif_prev'].dt.total_seconds()
events_sample.drop(['device_id_prev','date_dif_prev','date_prev'], axis='columns', inplace=True)
events_sample = events_sample.sort_values(by=['created'])
events_sample['user_agent']=np.nan
events_sample['kind']='event' #it is hashed

#average time
events_sample['hora'] = events_sample['created'].dt.hour
events_sample['dia'] = events_sample['created'].dt.day
pivot_auctions = events_sample.groupby(['dia','hora']).size().reset_index()
pivot_auctions.columns = ['dia','hora', 'hour_day_dist']
max_auctions = pivot_auctions['hour_day_dist'].max().max()
pivot_auctions['hour_day_dist'] = pivot_auctions['hour_day_dist']/max_auctions
display(pivot_auctions.head(5))
events_sample = pd.merge(events_sample, pivot_auctions, on=['dia','hora'], how='left')
events_sample.drop(columns=['hora','dia'], inplace=True)



,dia,hora,hour_day_dist
0,18,0,0.302979
1,18,1,0.292766
2,18,2,0.340426
3,18,3,0.379574
4,18,4,0.213617


In [14]:
events_sample.head()

,index,created,event_id,ref_type,ref_hash,application_id,attributed,device_os_version,device_brand,device_model,...,event_uuid,carrier,kind,device_os,wifi,connection_type,in_seconds,status_censored,last_seen,hour_day_dist
0,143310,2019-04-18 00:00:03.523,1,1891515180541284343,7010373312409084835,210,False,NaN,NaN,NaN,...,47fff218-65b9-4a73-9cf3-60ce3d9b9ba0,NaN,event,NaN,False,NaN,39.503,True,3.523,0.302979
1,68409,2019-04-18 00:00:39.020,15,1891515180541284343,4382914005843137510,302,False,NaN,5.645692e+18,1.697802e+18,...,e5513632-bca3-430d-a90a-6dad87322f97,NaN,event,NaN,True,NaN,92.262,True,39.020,0.302979
2,1329714,2019-04-18 00:00:43.026,1,1891515180541284343,7010373312409084835,210,False,NaN,NaN,NaN,...,32274059-b41d-482f-a63c-a2f979f44b22,NaN,event,NaN,False,NaN,59.284,True,39.503,0.302979
3,49641,2019-04-18 00:00:59.773,23,1891515180541284343,5142365577351731029,155,False,4.584084e+18,NaN,8.855249e+18,...,6aa5978f-489a-4937-90ea-bbb7cb4d6ed2,NaN,event,NaN,False,NaN,14.072,True,59.773,0.302979
4,54723,2019-04-18 00:01:09.029,23,1494519392962156891,5337742991722970079,116,False,4.821386e+18,NaN,6.208879e+18,...,8954d70e-0a1b-4a6a-a4e2-4397bf331fba,NaN,event,NaN,False,NaN,3.521,True,69.029,0.302979


In [15]:
#some features
install_cols = installs_sample.columns.tolist()
print(1)
#information about last installs and events
convertions = installs_sample[['ref_hash','created','application_id','user_agent','wifi','kind','in_seconds','status_censored','last_seen','hour_day_dist']].append(events_sample[['ref_hash', 'created', 'application_id','user_agent','wifi','kind','in_seconds','status_censored','last_seen','hour_day_dist']], ignore_index=True)
print(convertions.shape)
print(2)

#user agent
convertions['user_agent'] = np.where(convertions['user_agent'].str.contains('Android', regex=False),'Android',convertions['user_agent'])
convertions['user_agent'] = np.where(convertions['user_agent'].str.contains('Darwin', regex=False) | convertions['user_agent'].str.contains('iOS', regex=False),'iOS',convertions['user_agent'])
print(3)

# previus applications

app_id_1 = convertions[['application_id','created','ref_hash']].copy()
group_1 = pd.merge(convertions, app_id_1, on='ref_hash', how='inner')
group_1 = group_1.loc[(group_1['created_x']>group_1['created_y']) | group_1['created_y'].isnull()]
group_1['application_id'] = group_1['application_id_y']
group_1['created'] = group_1['created_x']
group_1.drop(columns=['application_id_y','created_x', 'application_id_x'], inplace=True)
group_1 = group_1.loc[group_1['created_y']>group_1['created']-pd.DateOffset(7)]
group_1 = pd.get_dummies(group_1,prefix_sep='=', dummy_na=True,columns=['application_id'],drop_first=True)
app_id_1_columns = []
for col in group_1.columns.tolist():
    if col.startswith('application_id'):
        app_id_1_columns.append(col)
group_1 = group_1.groupby(['created','ref_hash']).agg({col:'sum' for col in app_id_1_columns})
convertions = pd.merge(convertions, group_1, on=['created','ref_hash'], how='left')
convertions['curr_app_id'] = convertions['application_id']
convertions.drop(columns=['application_id'], inplace=True)
print(4)



convertions = pd.get_dummies(convertions,prefix_sep='=', dummy_na=True,columns=['curr_app_id','wifi', 'kind', 'user_agent'],drop_first=True)

convertions.fillna(value={'application_id=nan':1,'kind=nan':1,'curr_app_id=nan':1}, inplace=True)
print(6)
convertions.fillna(value={col:0 for col in app_id_1_columns}, inplace=True)
print(7)
convertions = convertions.astype({col:'int32' for col in app_id_1_columns})
print(8)
convertions.reset_index(inplace=True, drop=True)
print(9)


1
(71573, 10)
2
3
4
6
7
8
9


In [16]:
convertions.head()

,ref_hash,created,in_seconds,status_censored,last_seen,hour_day_dist,application_id=116,application_id=117,application_id=121,application_id=122,...,kind=reengagement,kind=registration_complete,kind=session begin,kind=sessionbegin,kind=sign in,kind=startsessionplayback,kind=terms_agree_split,kind=nan,user_agent=iOS,user_agent=nan
0,5142365577351731029,2019-04-18 00:00:23.583,259200.0,False,23.583,0.479167,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,6093271945346787320,2019-04-18 00:07:08.780,259200.0,False,428.780,0.479167,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2412759270791360354,2019-04-18 00:10:15.899,259200.0,False,615.899,0.479167,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1446104373415285607,2019-04-18 00:18:20.910,259200.0,False,1100.910,0.479167,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1816719220052514987,2019-04-18 00:23:22.760,259200.0,False,1402.760,0.479167,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
new_dtypes = {col: ('datetime64' if col=='created' else ('object' if col=='ref_hash' else ('float64' if col in ['in_seconds', 'last_seen', 'hour_day_dist'] else ('bool' if col in ['status_censored'] else 'int16')))) for col in convertions.columns.tolist()}
convertions = convertions.astype(new_dtypes, copy=False)
features = np.setdiff1d(convertions.columns, ['in_seconds', 'status_censored'] ).tolist()
corr = convertions[features].corr()
#remove uncorrelated columns (less than: -0.95)
corr = pd.DataFrame(np.tril(corr), columns=corr.columns, index=corr.index) #triangular inferior
removed_corr = corr.loc[(corr<-0.95).any()].index.tolist()
print(removed_corr)
corr_columns = np.setdiff1d(convertions.columns, removed_corr ).tolist()
if not('created' in corr_columns):
    corr_columns.append('created')
if not('ref_hash' in corr_columns):
    corr_columns.append('ref_hash')
if not('in_seconds' in corr_columns):
    corr_columns.append('in_seconds')
if not('last_seen' in corr_columns):
    corr_columns.append('last_seen')
if not('status_censored' in corr_columns):
    corr_columns.append('status_censored')
if not('hour_day_dist' in corr_columns):
    corr_columns.append('hour_day_dist')

convertions = convertions[corr_columns]
new_dtypes = {col: ('datetime64' if col=='created' else ('object' if col=='ref_hash' else ('float64' if col in ['in_seconds', 'last_seen', 'hour_day_dist'] else ('bool' if col in ['status_censored'] else 'int16')))) for col in convertions.columns.tolist()}
convertions.to_csv('data/convertions_merged_05.csv', index=False) 
new_dtypes


{'application_id=116': 'int16',
 'application_id=117': 'int16',
 'application_id=121': 'int16',
 'application_id=122': 'int16',
 'application_id=123': 'int16',
 'application_id=124': 'int16',
 'application_id=126': 'int16',
 'application_id=128': 'int16',
 'application_id=13': 'int16',
 'application_id=133': 'int16',
 'application_id=135': 'int16',
 'application_id=136': 'int16',
 'application_id=14': 'int16',
 'application_id=140': 'int16',
 'application_id=145': 'int16',
 'application_id=147': 'int16',
 'application_id=148': 'int16',
 'application_id=149': 'int16',
 'application_id=150': 'int16',
 'application_id=154': 'int16',
 'application_id=155': 'int16',
 'application_id=157': 'int16',
 'application_id=158': 'int16',
 'application_id=159': 'int16',
 'application_id=16': 'int16',
 'application_id=161': 'int16',
 'application_id=163': 'int16',
 'application_id=164': 'int16',
 'application_id=167': 'int16',
 'application_id=170': 'int16',
 'application_id=178': 'int16',
 'applicati

[]